# Monte Carlo & Adjoint-Mode Greeks  
**Jackson Pfaff – May 2025**

---
# 1. Monte Carlo Setup (One-Step GBM)

We simulate $N$ correlated assets over the interval $[0,T]$ in a single log-Euler step:

1. **Correlation and normal draws**  
   Let $\Sigma \in \mathbb{R}^{N \times N}$ be the assets’ correlation matrix.  
   Compute its lower-triangular Cholesky factor $L$ so that $LL^\top = \Sigma$.  
   Draw:

   $$
   Z \sim \mathcal{N}(\mathbf{0}, I_N), \quad Y = LZ \quad (\text{then } \mathrm{Cov}[Y] = \Sigma)
   $$

2. **Log-price increment**  
   For each asset $i = 1, \dots, N$, compute:

   $$
   \text{drift}_i = \left( r - \tfrac{1}{2} \sigma_i^2 \right) T, \quad
   \text{diffusion}_i = \sigma_i \sqrt{T} Y_i
   $$

3. **Terminal price**  
   Define the log-argument:

   $$
   G_i = \ln S_{0,i} + \text{drift}_i + \text{diffusion}_i
   $$

   Then:

   $$
   S_i(T) = e^{G_i}
   $$

4. **Worst-of payoff**  
   $$
   S^* = \min_{1 \le i \le N} S_i(T), \quad
   A = S^* - K, \quad
   h = \max(A, 0), \quad
   D = e^{-rT}, \quad
   P = D h
   $$

5. **Monte Carlo estimator**  
   Over $M$ independent paths:

   $$
   \widehat{V} = \frac{1}{M} \sum_{m=1}^M P^{(m)} =
   \frac{e^{-rT}}{M} \sum_{m=1}^M \left( \min_i S_i^{(m)}(T) - K \right)^+
   $$

---

# 2. Adjoint-Mode Delta

We wish to compute the pathwise sensitivity $\Delta_i = \partial P / \partial S_{0,i}$ via reverse-mode (adjoint) differentiation.

1. **Initialize**  
   $\bar{P} = 1$

2. **Back through discount**  
   $P = D h$ gives:

   $$
   \bar{h} = D \bar{P}, \quad
   \bar{D} = h \bar{P}
   $$

3. **Back through ReLU payoff**  
   $h = \max(A, 0)$ gives:

   $$
   \bar{A} = \mathbb{1}_{\{A > 0\}} \bar{h}
   $$

4. **Back through subtraction**  
   $A = S^* - K$ gives:

   $$
   \bar{S}^* = \bar{A}, \quad
   \bar{K} = -\bar{A}
   $$

5. **Back through minimum**  
   $S^* = \min_i S_i(T)$ gives, for each $i$:

   $$
   \bar{S}_i(T) = \mathbb{1}_{\{i = i^*\}} \bar{S}^*, \quad
   i^* = \operatorname*{arg\,min}_i S_i(T)
   $$

6. **Back through exponential**  
   $S_i(T) = e^{G_i}$ gives:

   $$
   \bar{G}_i = S_i(T) \bar{S}_i(T)
   $$

7. **Back through log-Euler step**  
   $G_i = \ln S_{0,i} + (r - \tfrac{1}{2} \sigma_i^2)T + \sigma_i \sqrt{T} Y_i$ gives:

   $$
   \bar{S}_{0,i} = \frac{1}{S_{0,i}} \bar{G}_i
   $$

Hence the pathwise Delta is:

$$
\boxed{
\Delta_i = \frac{\partial P}{\partial S_{0,i}} =
e^{-rT} \mathbb{1}_{\{A > 0\}} \mathbb{1}_{\{i = i^*\}} \frac{S_i(T)}{S_{0,i}}
}
$$

---

# 3. Adjoint-Mode Vega

We now compute $\mathrm{Vega}_i = \partial P / \partial \sigma_i$ by continuing the reverse sweep:

8. **Back through log-Euler step ($\sigma$ branch)**  
From the full expression:

$$
G_i = \ln S_{0,i} + \left( r - \tfrac{1}{2} \sigma_i^2 \right) T + \sigma_i \sqrt{T} Y_i
$$

Differentiate with respect to $\sigma_i$:

$$
\frac{\partial G_i}{\partial \sigma_i} = -\sigma_i T + \sqrt{T} Y_i
$$

Then the adjoint is:

$$
\bar{\sigma}_i = \left( -\sigma_i T + \sqrt{T} Y_i \right) \bar{G}_i
$$

Substituting $\bar{G}_i = S_i(T) \bar{S}_i(T)$ and the definition of $Y_i = (LZ)_i$:

$$
\boxed{
\mathrm{Vega}_i = e^{-rT} \mathbb{1}_{\{A > 0\}} \mathbb{1}_{\{i = i^*\}}
S_i(T) \left( -\sigma_i T + \sqrt{T} (LZ)_i \right)
}
$$

---

This completes the in-depth adjoint derivations for both Delta and Vega.

---
## References

- Capriotti, L. (2010) *Fast Greeks by Algorithmic Differentiation*
- Ferguson, S. & Green, J. (2018) “Deeply Learning Derivatives,” 
